In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../_apikeys.env")
api_key = os.getenv("DoogieOpenaiKey")
os.environ['OPENAI_API_KEY'] = api_key

In [2]:
#from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings( model="text-embedding-ada-002" )
# /var/folders/2y/695wr9bj2hl6y2fvhrwrs_4m0000gn/T/ipykernel_29009/970561986.py:3:
# LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0.
# An updated version of the class exists in the :class:`~langchain-openai package and should be used instead.
# To use it run `pip install -U :class:`~langchain-openai`
# and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
query_result = embeddings.embed_query( "나는 배가 아주 많이 고픈것 같다. 그렇기 때문에 나는 식사를 할 것인지, 아니면 굶을 것인지 고민이다.")

In [7]:
print(query_result.__sizeof__())
print(len(query_result), query_result[:5])

12328
1536 [-0.0017008718568831682, -0.041273199021816254, 0.01529977098107338, -0.016488604247570038, -0.01847860775887966]


In [16]:
import pandas as pd

somedata = [ '저는 배가 고파요',
             '저기 배가 지나 가네요',
             '굶어서 허기가 지네요',
             '허기 워리라는 게임이 즐거워요',
             '스팀에서 재미있는 것 해야지',
             '스팀으로 연어구이 해 먹을 거야',
             ]
#'저기 있는 배가 맛있을 것 같아요'
df = pd.DataFrame(somedata, columns=['SampleText'])
df

,SampleText
0,저는 배가 고파요
1,저기 배가 지나 가네요
2,굶어서 허기가 지네요
3,허기 워리라는 게임이 즐거워요
4,스팀에서 재미있는 것 해야지
5,스팀으로 연어구이 해 먹을 거야


In [17]:
def get_embedding(text):
    return embeddings.embed_query(text)

df['EmbedVec'] = df.apply(lambda row: get_embedding(row.SampleText), axis=1)
df


,SampleText,EmbedVec
0,저는 배가 고파요,"[-0.016708848997950554, -0.021796436980366707,..."
1,저기 배가 지나 가네요,"[-0.004745062906295061, -0.025967486202716827,..."
2,굶어서 허기가 지네요,"[-0.006181030999869108, -0.0069507937878370285..."
3,허기 워리라는 게임이 즐거워요,"[-0.013682473450899124, -0.009784186258912086,..."
4,스팀에서 재미있는 것 해야지,"[-0.007050866261124611, -0.017686471343040466,..."
5,스팀으로 연어구이 해 먹을 거야,"[-0.0020729016978293657, -0.028583526611328125..."


In [18]:
for embedVecA in df.EmbedVec:
    print( len(embedVecA), embedVecA[:2] )

1536 [-0.016708848997950554, -0.021796436980366707]
1536 [-0.004745062906295061, -0.025967486202716827]
1536 [-0.006181030999869108, -0.0069507937878370285]
1536 [-0.013682473450899124, -0.009784186258912086]
1536 [-0.007050866261124611, -0.017686471343040466]
1536 [-0.0020729016978293657, -0.028583526611328125]


In [19]:
#print(df.head())
print(type(df))
for rowA in df.itertuples(index=False):
    print( rowA.SampleText, rowA.EmbedVec[:2] )

<class 'pandas.core.frame.DataFrame'>
저는 배가 고파요 [-0.016708848997950554, -0.021796436980366707]
저기 배가 지나 가네요 [-0.004745062906295061, -0.025967486202716827]
굶어서 허기가 지네요 [-0.006181030999869108, -0.0069507937878370285]
허기 워리라는 게임이 즐거워요 [-0.013682473450899124, -0.009784186258912086]
스팀에서 재미있는 것 해야지 [-0.007050866261124611, -0.017686471343040466]
스팀으로 연어구이 해 먹을 거야 [-0.0020729016978293657, -0.028583526611328125]


In [20]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    return dot(a, b) / (norm(a) * norm(b))

def calc_sim(df, queryValue):
    queryValue_embedding = get_embedding(queryValue)
    #df['Similarity'] = df.EmbedVec.apply( lambda row: cos_sim(row, queryValue_embedding))
    #df['Similarity'] = df.EmbedVec.apply( cos_sim, args=(queryValue_embedding,))
    #df['Similarity'] = df.EmbedVec.apply( lambda row: cos_sim(row, queryValue_embedding) )
    #df['Similarity'] = df.EmbedVec.apply( lambda row: cos_sim(np.array(row), np.array(queryValue_embedding) ) )
    #df['Similarity'] = df.apply( lambda row: cos_sim(np.array(row.EmbedVec), np.array(queryValue_embedding) ), axis=1)
    df['Similarity'] = df.apply( lambda row: cos_sim(row.EmbedVec, queryValue_embedding), axis=1)
    #df['Similarity'] = df.apply( lambda row: cos_sim(row.EmbedVec, queryValue_embedding), axis=1)

calc_sim(df, "아무것도 안 먹었더니 꼬르륵 소리가 나네")
df


,SampleText,EmbedVec,Similarity
0,저는 배가 고파요,"[-0.016708848997950554, -0.021796436980366707,...",0.812257
1,저기 배가 지나 가네요,"[-0.004745062906295061, -0.025967486202716827,...",0.808944
2,굶어서 허기가 지네요,"[-0.006181030999869108, -0.0069507937878370285...",0.836768
3,허기 워리라는 게임이 즐거워요,"[-0.013682473450899124, -0.009784186258912086,...",0.785857
4,스팀에서 재미있는 것 해야지,"[-0.007050866261124611, -0.017686471343040466,...",0.788797
5,스팀으로 연어구이 해 먹을 거야,"[-0.0020729016978293657, -0.028583526611328125...",0.826998


In [21]:
def get_top3(df):
    df_top3 = df.sort_values(by=['Similarity'], ascending=False).head(3)
    return df_top3

result_top3 = get_top3(df)
result_top3

,SampleText,EmbedVec,Similarity
2,굶어서 허기가 지네요,"[-0.006181030999869108, -0.0069507937878370285...",0.836768
5,스팀으로 연어구이 해 먹을 거야,"[-0.0020729016978293657, -0.028583526611328125...",0.826998
0,저는 배가 고파요,"[-0.016708848997950554, -0.021796436980366707,...",0.812257
